In [65]:

import sys
from py_ecc.bn128 import G1, G2, G12, add, multiply, curve_order as p, pairing, eq, neg, Z1, FQ
import random
sys.path.append('../hw_1')
from pedersen_commitment import generate_n_random_points, pedersen_commitment

In [66]:
G = generate_n_random_points("goblin-plonk", 1)[0]
G_vec = [(FQ(6286155310766333871795042970372566906087502116590250812133967451320632869759), FQ(2167390362195738854837661032213065766665495464946848931705307210578191331138)),
     (FQ(6981010364086016896956769942642952706715308592529989685498391604818592148727), FQ(8391728260743032188974275148610213338920590040698592463908691408719331517047)),
     (FQ(15884001095869889564203381122824453959747209506336645297496580404216889561240), FQ(14397810633193722880623034635043699457129665948506123809325193598213289127838)),
     (FQ(6756792584920245352684519836070422133746350830019496743562729072905353421352), FQ(3439606165356845334365677247963536173939840949797525638557303009070611741415))]
H = (FQ(11573005146564785208103371178835230411907837176583832948426162169859927052980), FQ(895714868375763218941449355207566659176623507506487912740163487331762446439))
H_vec = [(FQ(13728162449721098615672844430261112538072166300311022796820929618959450231493), FQ(12153831869428634344429877091952509453770659237731690203490954547715195222919)),
    (FQ(17471368056527239558513938898018115153923978020864896155502359766132274520000), FQ(4119036649831316606545646423655922855925839689145200049841234351186746829602)),
    (FQ(8730867317615040501447514540731627986093652356953339319572790273814347116534), FQ(14893717982647482203420298569283769907955720318948910457352917488298566832491)),
    (FQ(419294495583131907906527833396935901898733653748716080944177732964425683442), FQ(14467906227467164575975695599962977164932514254303603096093942297417329342836))]

B = generate_n_random_points("whatsup", 1)[0]

In [67]:
class VectorPolynomial:
    def __init__(self, coefficients):
        if coefficients:
            dim = len(coefficients[0])
            if not all(len(v) == dim for v in coefficients):
                raise ValueError("All coefficient vectors must have the same dimension")
        
        self.coefficients = coefficients
        self.degree = len(coefficients) - 1 if coefficients else -1
        
    def evaluate(self, x):
        if not self.coefficients:
            return []
            
        result = [0] * len(self.coefficients[0])
        x_power = 1
        
        for coeff_vector in self.coefficients:
            term = [(x_power * c) % p for c in coeff_vector]
            result = [(r + t) % p for r, t in zip(result, term)]
            x_power = (x_power * x) % p
            
        return result
    
    def __str__(self):
        if not self.coefficients:
            return "0"
            
        terms = []
        for power, vector in enumerate(self.coefficients):
            if power == 0:
                terms.append(f"{vector}")
            elif power == 1:
                terms.append(f"{vector}x")
            else:
                terms.append(f"{vector}x^{power}")
                
        return " + ".join(terms)


In [68]:
class Polynomial:
    def __init__(self, coefficients):
        self.coefficients = coefficients
        self.degree = len(coefficients) - 1 if coefficients else -1

    def evaluate(self, x):
        result = 0
        for i, coef in enumerate(self.coefficients):
            result = (result + (coef * x**i) % p) % p
        return result


In [69]:
def calculate_inner_product(a, b):
    if len(a) != len(b):
        raise ValueError("Lists must be of the same length")
    return sum((x * y) % p for x, y in zip(a, b)) % p

In [70]:
def calculate_inner_product_with_ec(a, G):
  result = Z1
  for index, val in enumerate(a):
    result = add(result, multiply(G[index], int(val)))
  
  return result

In [71]:
def generate_inner_product_commitment(a, b):
  v = calculate_inner_product(a, b)
  s_l = [random.randint(1, 1000) % p for _ in range(len(a))]
  s_r = [random.randint(1, 1000) % p for _ in range(len(b))]

  l = VectorPolynomial([a, s_l])
  r = VectorPolynomial([b, s_r])

  t_coeffs = [v, (calculate_inner_product(a, s_r) + calculate_inner_product(b, s_l)) % p, calculate_inner_product(s_r, s_l) % p]
  t = Polynomial(t_coeffs)
  
  alpha = random.randint(0, 10000) % p
  A = add(add(calculate_inner_product_with_ec(a, G_vec), calculate_inner_product_with_ec(b, H_vec)), multiply(B, int(alpha)))

  beta = random.randint(1, 10000) % p
  S = add(add(calculate_inner_product_with_ec(s_l, G_vec), calculate_inner_product_with_ec(s_r, H_vec)), multiply(B, int(beta)))

  gamma = random.randint(0, 10000) % p
  V = add(multiply(G, int(v)), multiply(B, int(gamma)))

  tau_1 = random.randint(1, 1000000) % p
  t_1_scalar = calculate_inner_product(a, s_r) + calculate_inner_product(b, s_l)
  T_1 = add(multiply(G, int(t_1_scalar)), multiply(B, int(tau_1)))
  
  tau_2 = random.randint(0, 100000) % p
  T_2 = add(multiply(G, int(calculate_inner_product(s_l, s_r))), multiply(B, int(tau_2)))

  return [(A, S, V, T_1, T_2), (l, r, t), (alpha, beta, gamma, tau_1, tau_2)]
  


In [72]:
a = [20, 14, 40, 10]
b = [34, 45, 60, 10]
[(A, S, V, T_1, T_2), (l, r, t), (alpha, beta, gamma, tau_1, tau_2)] = generate_inner_product_commitment(a, b)
u = random.randint(1, 10000)


In [73]:
def generate_inner_product_proof(u):
  l_u = l.evaluate(u)
  r_u = r.evaluate(u)
  t_u = t.evaluate(u)

  pi_lr = alpha + beta * u
  pi_t = gamma + tau_1 * u + tau_2 * u * u

  return (l_u, r_u, t_u, pi_lr, pi_t)

In [74]:
(l_u, r_u, t_u, pi_lr, pi_t) = generate_inner_product_proof(u)

In [75]:
from functools import reduce
import random

def random_element():
    return random.randint(0, p)

def add_points(*points):
    return reduce(add, points, Z1)

def vector_commit(points, scalars):
    return reduce(add, [multiply(P, i) for P, i in zip(points, scalars)], Z1)

def mod_inner(a, b, p):
    return sum((x * y) % p for x, y in zip(a, b)) % p

def mod_scalar_mul(arr, scalar, p):
    return [(x * scalar) % p for x in arr]

def mod_vec_add(a, b, p):
    return [(x + y) % p for x, y in zip(a, b)]

def mod_vec_mul(a, b, p):
    return [(x * y) % p for x, y in zip(a, b)]

def fold(scalar_vec, u):
    i = 0
    vec = []
    while i < len(scalar_vec):
        vec.append((mod_inner([scalar_vec[i]], [u], p) + mod_inner([scalar_vec[i+1]], [pow(u, -1, p)], p)) % p)
        i += 2
    
    assert len(vec) == len(scalar_vec) / 2
    return vec

def fold_points(point_vec, u):
    i = 0
    vec = []
    while i < len(point_vec):
        vec.append(add_points(multiply(point_vec[i], u), multiply(point_vec[i+1], pow(u, -1, p))))
        i += 2
    
    return vec

def compute_secondary_diagonal(G_vec, a):
    R = Z1
    L = Z1

    for i in range(len(a)):
        if i % 2 == 0:
            R = add_points(R, multiply(G_vec[i], a[i+1]))
        else:
            L = add_points(L, multiply(G_vec[i], a[i-1]))

    return L, R

def compute_secondary_diagonal_scalar(b, a):
    R = 0
    L = 0

    for i in range(len(a)):
        if i % 2 == 0:
            R = (R + (b[i] * a[i+1] % p)) % p
        else:
            L = (L + (b[i] * a[i-1] % p)) % p

    return L, R

def verify(a, b, b_inv, P, G_vec, H_vec, Q):

    assert len(a) == len(b) == len(b_inv) == len(G_vec) == len(H_vec), "vectors must be of same length"

    if len(a) > 1:
        # Compute L and R    
        L_a, R_a = compute_secondary_diagonal(G_vec, a)
        L_b, R_b = compute_secondary_diagonal(H_vec, b)
        L_q, R_q = compute_secondary_diagonal_scalar(b_inv, a)

        L = add_points(L_a, L_b, multiply(Q, L_q))
        R = add_points(R_a, R_b, multiply(Q, R_q))

        # Verifier provided randomness
        u = random_element()

        # Compute P'
        Pprime = add_points(multiply(L, pow(u, 2, p)), P, multiply(R, pow(u, -2, p)))

        # Fold
        aprime = fold(a, u)
        bprime = fold(b, u)
        bprime_inv = fold(b_inv, pow(u, -1, p))

        Gprime = fold_points(G_vec, pow(u, -1, p))
        Hprime = fold_points(H_vec, pow(u, -1, p))

        print(vector_commit(Gprime, aprime))

        return verify(aprime, bprime, bprime_inv, Pprime, Gprime, Hprime, Q)
    else:
        return eq(P, add_points(vector_commit(G_vec, a), vector_commit(H_vec, b), multiply(Q, mod_inner(a, b_inv, p))))


In [78]:
def verify_proof(commitments, proofs, u):
	(A, S, V, T_1, T_2) = commitments
	(l_u, r_u, t_u, pi_lr, pi_t) = proofs

	C = add(add(vector_commit(G_vec, l_u), vector_commit(H_vec, r_u)), multiply(H, t_u))
	verification = verify(l_u, r_u, r_u, C, G_vec, H_vec, H)

	if not verification:
		raise "Invalid Proof"
	
	eqn_3_lhs = add(multiply(G, int(t_u)), multiply(B, int(pi_t)))
	eqn_3_rhs = add(V, add(multiply(T_1, int(u)), multiply(T_2, int(u * u))))

	if eqn_3_lhs != eqn_3_rhs:
		raise "Invalid Proof"
	
	return True

In [79]:
isVerified = verify_proof((A, S, V, T_1, T_2), (l_u, r_u, t_u, pi_lr, pi_t), u )

print(isVerified)

(7978241974260838914998214688443281812326164544511182494288408518765421646676, 21028513001143792772578560132868650259188143525293308827423302165042499503522)
(6596527956429189328220751395769266092455957212448672075761908544639477060875, 14795215192954967135252359262767867131124270172094756117720378813039308957306)
True
